<a href="https://colab.research.google.com/github/Yuji-github/X_ray-Oject-Detection/blob/main/Slim_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/YOLO_Custom

/content/drive/MyDrive/YOLO_Custom


**Create Data Folder at YOLO_Custom**

In [ ]:
%mkdir data # if tain.txt has Data/obj/****.jpg, thne folder name is Data

**Make Sure Files Folder contains [ _darknet.txt, train.txt, and valid.txt [link text ]**

In [ ]:
%cp Files/_darknet.txt Train/_darknet.labels
%cp Files/train.txt data/train.txt
%cp Files/vaild.txt data/valid.txt

**The below step takes a lot of time** 

In [ ]:
%cp Train/_darknet.labels data/obj.names # create files for learning, make sure correct location and create data folder for each custome detection
%mkdir data/obj # if data folder is not empty, make new data1, data2 and so on and change the folder names

%cp Train/*.jpg data/obj/
%cp Valid/*.jpg data/obj/

%cp Train/*.txt data/obj/
%cp Valid/*.txt data/obj/

with open('data/obj.data', 'w') as out:
  out.write('classes = 4\n')
  out.write('train = data/train.txt\n')
  out.write('valid = data/valid.txt\n')
  out.write('names = data/obj.names\n')
  out.write('backup = backup/')

**Giving the permission to access darknet from /content/drive/MyDrive/YOLO_Custom If permission error occurs**

In [ ]:
!chmod +x darknet/darknet # if the location is different try !chmod +x ./darknet

Check lists: <br> Did you create/modify a cfg file? <br> Are the images size divided by 32? <br> Do you have all items in the Data folder? <br> Did you compile YOLO %make if you installed YOLO as the first time? <br> Do you have a backup folder? <br> Is the labels folder in the Data foler?

**Training: GPU MUST BE ON**

In [ ]:
!darknet/darknet detector train data/obj.data darknet/cfg/yolov4_custom1.cfg custom_weight/yolov4.conv.137 -dont_show #train data/data.data

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov4_custom1
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 

**For testing, must change the batch and subdivisons**

In [ ]:
!sed -i 's/batch=64/batch=1/' cfg/yolov4_custom1.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' cfg/yolov4_custom1.cfg

**Evaluation**

In [ ]:
#define utility function
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
%cd /content/drive/MyDrive/YOLO_Custom

**Check Weight**

In [ ]:
!ls backup

In [ ]:
#/test has images that we can test our detector on
import os
test_images = [f for f in os.listdir('Test') if f.endswith('.jpg')]
import random
img_path = "Test/" + random.choice(test_images);

#test out our detector!
!darknet/darknet detector test data/obj.data darknet/cfg/yolov4_custom.cfg backup/yolov4_custom_last.weights {img_path} -dont-show
imShow('/content/drive/MyDrive/YOLO_Custom/predictions.jpg')